# Greedy Entry Removal Algorithm (GER)

In [1]:
import numpy as np

### The Entry Selection Procedure for the GER algorithm

Step 1, 2, ..., n are the step labels in the Microsoft Word document "construct simpler versions of the GER algorithm (2023-2-5).docx".

In [7]:
 def GER_entry_choosing_procedure(residue_matrix_R, score_parameter_z):
    R_row_num = residue_matrix_R.shape[0]
    R_col_num = residue_matrix_R.shape[1]
    chosen_entry_pos_each_col = np.array([[-R_row_num - 1] * R_col_num])
    
    # Step 1
    max_entry_M = np.amax(residue_matrix_R)
    
    # Step 2
    # the col dimension should be max_entry_M + 1 because index starts at 0 but not 1.
    occurrence_count_Li = np.zeros((R_col_num, 1 + max_entry_M), dtype=int)
    occurrence_indication_Ki = np.zeros((R_col_num, 1 + max_entry_M), dtype=int)
    
    for col_counter in range(R_col_num):
        for row_counter in range(R_row_num):
            entry_val_a = residue_matrix_R[row_counter, col_counter]
            if entry_val_a > 0:
                occurrence_indication_Ki[col_counter, entry_val_a] = 1
                occurrence_count_Li[col_counter, entry_val_a] += 1
    
    # Step 3
    max_entry_each_col_Mi = np.amax(residue_matrix_R, axis=0)
    upper_bound_B = np.amin(max_entry_each_col_Mi)
    
    # Step 4
    occurrence_all_columns_K = np.sum(occurrence_indication_Ki, axis=0)
    
    # Step 5
    first_B_vals_of_K = occurrence_all_columns_K[0:upper_bound_B + 1] # the value at index 0 should be ignored.
    most_occurred_entries_v = find_most_occurred_entries(first_B_vals_of_K, upper_bound_B)
    # most_occurred_entries_v is a list.
    
    # Step 6
    chosen_entry_pos_each_col_for_each_v = {}
    indicator_array_for_selected_columns_for_each_v = {}
    occurrence_count_Li_for_each_v = {}
    occurrence_indication_Ki_for_each_v = {}
    cumulative_occurrence_indication_Ktoj_cumu_for_each_v = {}
    selection_score_u_for_each_v = {}
    
    for v in most_occurred_entries_v:
        chosen_entry_pos_each_col_for_each_v[v] = np.array([[-R_row_num - 1] * R_col_num])
        indicator_array_for_selected_columns_for_each_v[v] = np.zeros(R_col_num, dtype=int)
        occurrence_count_Li_for_each_v[v] = occurrence_count_Li.copy()
        occurrence_indication_Ki_for_each_v[v] = occurrence_indication_Ki.copy()
        cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] = np.zeros(1 + max_entry_M)
    
    for v in most_occurred_entries_v:
        for col_counter_i in range(R_col_num):
            if occurrence_indication_Ki_for_each_v[v][col_counter_i, v] == 1:
                q = find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, col_counter_i, v)
                chosen_entry_pos_each_col_for_each_v[v][0, col_counter_i] = q
                indicator_array_for_selected_columns_for_each_v[v][col_counter_i] = 1
                
                occurrence_count_Li_for_each_v[v][col_counter_i, v] -= 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, v] == 0:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, v] = 0
                cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] += \
                occurrence_indication_Ki_for_each_v[v][col_counter_i, :]
    
        for col_counter_i in range(R_col_num):
            if indicator_array_for_selected_columns_for_each_v[v][col_counter_i] == 0:
                list_of_candidate_entry_vals = \
                all_entry_vals_greater_than_val(residue_matrix_R, R_row_num, col_counter_i, v)
                
                occurrences_in_cumulative_Ktoj_cumu = {}
                for val in list_of_candidate_entry_vals:
                    occurrences_in_cumulative_Ktoj_cumu[val] = \
                    cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v][val - v]
            
                greatest_occurrence = -1
                entry_with_greatest_occurrence = -1
            
                for val in list_of_candidate_entry_vals:
                    if greatest_occurrence < occurrences_in_cumulative_Ktoj_cumu[val]:
                        greatest_occurrence = occurrences_in_cumulative_Ktoj_cumu[val]
                        entry_with_greatest_occurrence = val
                
                chosen_entry_pos_each_col_for_each_v[v][0, col_counter_i] = \
                find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, 
                                                         col_counter_i, entry_with_greatest_occurrence)
                
                indicator_array_for_selected_columns_for_each_v[v][col_counter_i] = 1
                
                occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] -= 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] == 0:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] = 0
                
                occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence - v] += 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence - v] == 1:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, 
                                                           entry_with_greatest_occurrence - v] = 1
                
                cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] += \
                occurrence_indication_Ki_for_each_v[v][col_counter_i, :]
        
        selection_score_u_for_each_v[v] = \
        compute_selection_score(cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v], 
                                max_entry_M, score_parameter_z)
    
    # step 7
    most_favourable_v_star = \
    choose_most_favourable_v_star(most_occurred_entries_v, selection_score_u_for_each_v)
    
    chosen_entry_xi = most_favourable_v_star
    chosen_entry_pos_each_col = chosen_entry_pos_each_col_for_each_v[most_favourable_v_star]
                
    return chosen_entry_xi, chosen_entry_pos_each_col
    # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.

#### [1] The following function is utilised in step 5 of GER_entry_choosing_procedure.

In [2]:
# The input first_B_vals_of_K of this function is a 1D numpy array of size
# (upper_bound_B + 1), and the entry at index zero of this array is to be ignored.
# The output of this function is a list.
def find_most_occurred_entries(first_B_vals_of_K, upper_bound_B):
    most_occurred_entries_v = []
    greatest_occurrence = -1
    
    for v in range(1, upper_bound_B + 1):
        if first_B_vals_of_K[v] > greatest_occurrence:
            greatest_occurrence = first_B_vals_of_K[v]
            most_occurred_entries_v = [v]
        elif first_B_vals_of_K[v] == greatest_occurrence:
            most_occurred_entries_v.append(v)
    
    return most_occurred_entries_v

Test the function find_most_occurred_entries:

In [17]:
test_ndarray_1 = [0, 0, 0, 2, 0, 0, 3, 1, 0, 1, 3]
test_B_1 = 10

find_most_occurred_entries(test_ndarray_1, test_B_1)

[6, 10]

In [18]:
test_ndarray_2 = [0, 1, 0, 1, 1, 0, 2, 0, 2]
test_B_2 = 8

find_most_occurred_entries(test_ndarray_2, test_B_2)

[6, 8]

In [19]:
test_ndarray_3 = [0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
test_B_3 = 9

find_most_occurred_entries(test_ndarray_3, test_B_3)

[3, 5, 7, 9]

#### [2] The following function is utilised in step 6 of GER_entry_choosing_procedure to find the position of a specific value in a particular column of the residue matrix.

Description of the arguments:

residue_matrix_R: the residue matrix to be searched (2D numpy ndarray).

R_row_num: the number of rows in residue_matrix_R.

col: the index of the column to be searched.

val: the value to be searched.

It is assumed that val must be present in that specific column of residue_matrix_R.

In [3]:
def find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, col, val):
    for row_counter in range(R_row_num):
        if val == residue_matrix_R[row_counter, col]:
            return row_counter

#### [3] The following function is utilised in step 6 of GER_entry_choosing_procedure to find all the entries in a specific column that are greater than a specified value.

Description of the arguments:

residue_matrix_R: the residue matrix to be searched (2D numpy ndarray).

R_row_num: the number of rows in residue_matrix_R.

col: the index of the column to be searched.

val: the specified value.

It is assumed that val is not present in that specific column of residue_matrix_R.

In [4]:
def all_entry_vals_greater_than_val(residue_matrix_R, R_row_num, col, val):
    output_list_of_entry_vals = []
    
    for row_counter in range(R_row_num):
        if residue_matrix_R[row_counter, col] > val:
            output_list_of_entry_vals.append(residue_matrix_R[row_counter, col])
    
    return output_list_of_entry_vals

Test the function all_entry_vals_greater_than_val:

In [47]:
test_array = np.array([[38, 22], [7, 5], [26, 8], [9, 30]])
test_val = 30
all_entry_vals_greater_than_val(test_array, 4, 0, test_val)

[38]

#### [4] The following function is used in step 6 of GER_entry_choosing_procedure to compute the selection score.

In [5]:
# cumulative_occurrence_array is an array of size (1 + max_entry_M). The entry
# at index 0 is useless and should never be used.
def compute_selection_score(cumulative_occurrence_array, max_entry_M, score_parameter_z):
    selection_score = 0
    
    for v in range(1, max_entry_M + 1):
        if cumulative_occurrence_array[v] > 0:
            selection_score += score_parameter_z ** cumulative_occurrence_array[v]
    
    return selection_score

Test the function compute_selection_score.

In [50]:
test_array = np.array([0, 0, 0, 1, 0, 1, 2, 3, 4, 0])
test_max_entry = 9
test_score_parameter = 10

compute_selection_score(test_array, test_max_entry, test_score_parameter)

11120

#### [5] The following function is used in step 7 of the GER_entry_choosing_procedure.

Description of the input parameters:

1. most_occurred_entries_v: a list
2. selection_score_u_for_each_v: a dictionary with keys from most_occurred_entries_v. The value of each key is an int.


In [6]:
def choose_most_favourable_v_star(most_occurred_entries_v, selection_score_u_for_each_v):
    list_of_entries_with_max_score = []
    max_score_found_so_far = -1
    
    for v in most_occurred_entries_v:
        if max_score_found_so_far < selection_score_u_for_each_v[v]:
            max_score_found_so_far = selection_score_u_for_each_v[v]
            list_of_entries_with_max_score = [v]
        elif max_score_found_so_far == selection_score_u_for_each_v[v]:
            list_of_entries_with_max_score.append(v)
    
    return max(list_of_entries_with_max_score)

Test the function choose_most_favourable_v_star:

In [55]:
test_entries_list = [8, 54, 36, 77, 88]
test_dictionary = {36: 100, 54: 25, 8: 99, 77: 100, 88: 100}
choose_most_favourable_v_star(test_entries_list, test_dictionary)

88

### Test the function GER_entry_choosing_procedure

In [59]:
test1 = np.array([[2, 9], 
                  [8, 1]])
GER_entry_choosing_procedure(test1, 10)

(2, array([[0, 0]]))

In [60]:
test2 = np.array([[12, 45, 75,  3], 
                  [45,  4,  3, 61], 
                  [ 5,  5,  5, 67]])
GER_entry_choosing_procedure(test2, 10)

(5, array([[2, 2, 2, 1]]))

In [61]:
test3 = np.array([[1, 5, 6,  0], 
                  [4, 0, 2,  0], 
                  [5, 2, 0, 10], 
                  [0, 3, 2,  0]])
GER_entry_choosing_procedure(test3, 10)

(2, array([[1, 2, 1, 2]]))

### The GER function

In [8]:
def GER(PBN_matrix_P, score_parameter_z):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, chosen_entry_pos_each_col = \
    GER_entry_choosing_procedure(residue_matrix_R, score_parameter_z)
    # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_xi, chosen_entry_pos_each_col = \
        GER_entry_choosing_procedure(residue_matrix_R,  score_parameter_z)
        # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

### Outputs of GER on different small PBN matrices

#### The following function is used to permute the entries in each column of the three PBN matrices used to break SER 2.

In [ ]:
def permute_entries_in_each_col(input_PBN_matrix):
    num_of_rows, num_of_cols = input_PBN_matrix.shape
    permuted_PBN_matrix = np.zeros(input_PBN_matrix.shape, dtype=int)
    
    for col_counter in range(num_of_cols):
        rearranged_row_indices = np.random.permutation(num_of_rows)
        for row_counter in range(num_of_rows):
            permuted_PBN_matrix[row_counter, col_counter] = \
            input_PBN_matrix[rearranged_row_indices[row_counter], col_counter]
    
    return permuted_PBN_matrix

#### The following function is used to permute the columns of the three PBN matrices used to break SER 2.

In [ ]:
def permute_the_cols(input_PBN_matrix):
    num_of_rows, num_of_cols = input_PBN_matrix.shape
    permuted_PBN_matrix = np.zeros(input_PBN_matrix.shape, dtype=int)
    rearranged_col_indices = np.random.permutation(num_of_cols)
    
    for col_counter in range(num_of_cols):
        permuted_PBN_matrix[:, col_counter] = input_PBN_matrix[:, rearranged_col_indices[col_counter]]
    
    return permuted_PBN_matrix

#### [1] PBN Matrix from Default Data

In [11]:
test_PBN_matrix_3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                              [ 0, 15,  0,  0,  0,  8,  0,  0], 
                              [ 0,  8, 40, 25, 25,  0, 67,  0], 
                              [ 0,  0,  0,  0, 38,  0,  0,  0],
                              [14, 31,  0, 50, 12, 13, 33,  6],
                              [29, 31, 20,  0, 25, 29,  0, 39],
                              [ 0, 15, 30,  0,  0, 13,  0,  0],
                              [ 0,  0,  0, 25,  0, 33,  0, 55]])
GER(test_PBN_matrix_3, 2)

(11,
 [25, 8, 6, 6, 25, 2, 13, 7, 4, 3, 1],
 array([[0, 4, 2, 2, 2, 7, 4, 5],
        [4, 2, 0, 4, 3, 1, 4, 5],
        [4, 4, 2, 4, 4, 7, 2, 4],
        [0, 5, 2, 4, 4, 4, 2, 5],
        [0, 5, 6, 7, 5, 5, 2, 7],
        [5, 1, 0, 4, 3, 7, 2, 7],
        [5, 1, 5, 4, 3, 6, 2, 7],
        [5, 6, 5, 4, 3, 4, 2, 7],
        [5, 6, 6, 4, 3, 0, 2, 7],
        [5, 6, 2, 4, 3, 5, 2, 7],
        [0, 6, 6, 4, 3, 5, 2, 7]]))

#### [2] Nursing Home PBN Matrix

In [69]:
test_PBN_matrix_4 = np.array([[0, 6, 8, 1, 7, 5, 8, 5, 6, 5, 3, 4, 10, 2, 2, 0], 
                              [8, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 3], 
                              [2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 0], 
                              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 1, 1, 8, 2, 4, 2, 5, 0, 3, 2, 3,  0, 3, 4, 1],
                              [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 3,  0, 0, 2, 1],
                              [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,  0, 4, 2, 5],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 1, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0]
                             ])
GER(test_PBN_matrix_4, 10)

(7,
 [1, 1, 3, 2, 1, 1, 1],
 array([[ 2,  1,  1,  0,  8,  5,  4,  0,  1,  9,  4,  0,  0, 13,  0,  4],
        [ 2,  2,  4,  5,  0,  4,  4,  0,  2,  9,  4,  0,  0, 12,  0,  8],
        [ 1,  0,  0,  4,  0,  4,  0,  0,  0,  4,  0,  4,  0,  4,  4,  1],
        [ 1,  0,  0,  4,  4,  0,  0,  4,  0,  0,  9,  0,  0,  0,  8, 12],
        [ 1,  0,  0,  4,  0,  0,  0,  4,  0,  0, 12,  8,  0, 12,  4, 12],
        [ 1,  3,  0,  4,  0,  0,  0,  4,  8,  0, 12,  8,  0, 12, 12, 12],
        [ 1,  4,  0,  4,  0,  0,  0,  4,  9,  0, 12,  8,  0, 12, 12, 12]]))

#### [3] First PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 110.

##### Execute GER before permuting the PBN matrix.

In [10]:
x1 = 10
x2 = 12
x3 = 15
x4 = 19
x5 = 24
x6 = 30

test_PBN_matrix_5 = np.array([[   x1,    x1,    x1,    x1,    x1, x1+x2, x1+x2, x1+x2], 
                              [   x2,    x2,    x2,    x2,    x2,     0,     0,     0], 
                              [x3+x4, x3+x4,    x3,    x3,    x3,    x3,    x3, x3+x4], 
                              [    0,     0,    x4,    x4,    x4,    x4,    x4,     0],
                              [   x5, x5+x6, x5+x6, x5+x6, x5+x6,    x5,    x5,    x5],
                              [   x6,     0,     0,     0,     0,    x6,    x6,    x6],
                              [    0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0]])
test_PBN_matrix_5

array([[10, 10, 10, 10, 10, 22, 22, 22],
       [12, 12, 12, 12, 12,  0,  0,  0],
       [34, 34, 15, 15, 15, 15, 15, 34],
       [ 0,  0, 19, 19, 19, 19, 19,  0],
       [24, 54, 54, 54, 54, 24, 24, 24],
       [30,  0,  0,  0,  0, 30, 30, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [11]:
GER(test_PBN_matrix_5, 10)

(6,
 [19, 15, 12, 10, 30, 24],
 array([[2, 2, 3, 3, 3, 3, 3, 2],
        [2, 2, 2, 2, 2, 2, 2, 2],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [5, 4, 4, 4, 4, 5, 5, 5],
        [4, 4, 4, 4, 4, 4, 4, 4]]))

##### Let's permute the entries in the PBN matrix.

In [12]:
# test_PBN_matrix_5_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_5))
test_PBN_matrix_5_permuted = np.array([[12, 30, 22, 10, 10, 15, 54, 34],
                                       [10, 24, 19, 54, 30,  0,  0,  0],
                                       [54, 15,  0, 12, 12,  0,  0, 30],
                                       [ 0,  0, 24, 15, 24, 19, 10,  0],
                                       [ 0,  0,  0,  0, 34, 10, 12,  0],
                                       [19,  0, 15,  0,  0, 12,  0, 22],
                                       [15, 19,  0, 19,  0, 54,  0, 24],
                                       [ 0, 22, 30,  0,  0,  0, 34,  0]])

##### Execute GER on the permuted PBN matrix.

In [14]:
GER(test_PBN_matrix_5_permuted, 10)

(6,
 [19, 15, 12, 10, 30, 24],
 array([[5, 6, 1, 6, 4, 3, 7, 0],
        [6, 2, 5, 3, 4, 0, 7, 0],
        [0, 7, 0, 2, 2, 5, 4, 5],
        [1, 7, 0, 0, 0, 4, 3, 5],
        [2, 0, 7, 1, 1, 6, 0, 2],
        [2, 1, 3, 1, 3, 6, 0, 6]]))

#### [4] Second PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 110.

##### Execute GER before permuting the PBN matrix.

In [15]:
x1 = 10
x2 = 12
x3 = 15
x4 = 19
x5 = 24
x6 = 30

test_PBN_matrix_6 = np.array([[   x1,    x1, x1+x2, x1+x2, x1+x2, x1+x3, x1+x3, x1+x3], 
                              [   x2,    x2,    x3,    x3,    x3,    x2,    x2,    x2], 
                              [   x3,    x3,     0,     0,     0,     0,     0,     0], 
                              [x4+x6, x4+x6, x4+x6, x4+x5, x4+x5, x4+x5,    x4,    x4],
                              [   x5,    x5,    x5,    x6,    x6,    x6,    x5,    x5],
                              [    0,     0,     0,     0,     0,     0,    x6,    x6],
                              [    0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0]])
test_PBN_matrix_6

array([[10, 10, 22, 22, 22, 25, 25, 25],
       [12, 12, 15, 15, 15, 12, 12, 12],
       [15, 15,  0,  0,  0,  0,  0,  0],
       [49, 49, 49, 43, 43, 43, 19, 19],
       [24, 24, 24, 30, 30, 30, 24, 24],
       [ 0,  0,  0,  0,  0,  0, 30, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [16]:
GER(test_PBN_matrix_6, 10)

(6,
 [30, 24, 19, 15, 12, 10],
 array([[3, 3, 3, 4, 4, 4, 5, 5],
        [4, 4, 4, 3, 3, 3, 4, 4],
        [3, 3, 3, 3, 3, 3, 3, 3],
        [2, 2, 1, 1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0]]))

##### Let's permute the entries in the PBN matrix.

In [17]:
# test_PBN_matrix_6_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_6))
test_PBN_matrix_6_permuted = np.array([[ 0,  0,  0, 49,  0, 43,  0, 49],
                                       [ 0, 30, 12,  0, 30,  0, 25,  0],
                                       [25,  0,  0, 15,  0, 22, 12, 15],
                                       [ 0,  0, 10, 24, 19,  0, 30,  0],
                                       [30, 43, 15,  0, 24, 15,  0,  0],
                                       [43,  0, 49,  0,  0,  0, 19, 24],
                                       [ 0, 22,  0, 22, 12, 30,  0, 12],
                                       [12, 15, 24,  0, 25,  0, 24, 10]])

##### Execute GER on the permuted PBN matrix.

In [19]:
GER(test_PBN_matrix_6_permuted, 10)

(6,
 [30, 24, 19, 15, 12, 10],
 array([[4, 1, 5, 0, 1, 6, 3, 0],
        [5, 4, 7, 3, 4, 0, 7, 5],
        [5, 4, 5, 0, 3, 0, 5, 0],
        [2, 7, 4, 2, 7, 4, 1, 2],
        [7, 6, 1, 6, 6, 2, 2, 6],
        [2, 6, 3, 6, 7, 2, 1, 7]]))

#### [5] Third PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 265.

##### Execute GER before permuting the PBN matrix.

In [20]:
x1 = 37
x2 = 26
x3 = 17
x4 = 9
x5 = 59
x6 = 49
x7 = 39
x8 = 29

test_PBN_matrix_7 = np.array([[   x1,    x1,    x1,    x1, x1+x2, x1+x2, x1+x2, x1+x2, x1+x3, x1+x3, x1+x3, x1+x3, x1+x4, x1+x4, x1+x4, x1+x4], 
                              [   x2,    x2,    x2,    x2,    x3,    x3,    x3,    x3,    x2,    x2,    x2,    x2,    x2,    x2,    x2,    x2], 
                              [   x3,    x3,    x3,    x3,    x4,    x4,    x4,    x4,    x4,    x4,    x4,    x4,    x3,    x3,    x3,    x3], 
                              [   x4,    x4,    x4,    x4,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [x5+x6, x5+x6, x5+x7, x5+x8, x5+x7, x5+x8, x5+x6,    x5, x5+x7, x5+x8, x5+x6,    x5, x5+x7, x5+x8,    x5,    x5],
                              [    0,     0,    x6,    x6,    x6,    x6,     0,    x6,    x6,    x6,     0,    x6,    x6,    x6,    x6,    x6],
                              [   x7,    x7,     0,    x7,     0,    x7,    x7,    x7,     0,    x7,    x7,    x7,     0,    x7,    x7,    x7],
                              [   x8,    x8,    x8,     0,    x8,     0,    x8,    x8,    x8,     0,    x8,    x8,    x8,     0,    x8,    x8],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                             ])
test_PBN_matrix_7

array([[ 37,  37,  37,  37,  63,  63,  63,  63,  54,  54,  54,  54,  46,
         46,  46,  46],
       [ 26,  26,  26,  26,  17,  17,  17,  17,  26,  26,  26,  26,  26,
         26,  26,  26],
       [ 17,  17,  17,  17,   9,   9,   9,   9,   9,   9,   9,   9,  17,
         17,  17,  17],
       [  9,   9,   9,   9,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [108, 108,  98,  88,  98,  88, 108,  59,  98,  88, 108,  59,  98,
         88,  59,  59],
       [  0,   0,  49,  49,  49,  49,   0,  49,  49,  49,   0,  49,  49,
         49,  49,  49],
       [ 39,  39,   0,  39,   0,  39,  39,  39,   0,  39,  39,  39,   0,
         39,  39,  39],
       [ 29,  29,  29,   0,  29,   0,  29,  29,  29,   0,  29,  29,  29,
          0,  29,  29],
       [  0,   0,   0,   0,   

In [21]:
GER(test_PBN_matrix_7, 10)

(8,
 [49, 39, 59, 29, 26, 17, 37, 9],
 array([[6, 6, 7, 7, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 7, 7],
        [8, 8, 6, 8, 6, 8, 8, 8, 6, 8, 8, 8, 6, 8, 8, 8],
        [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
        [9, 9, 9, 6, 9, 6, 9, 9, 9, 6, 9, 9, 9, 6, 9, 9],
        [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0]]))

##### Let's permute the entries in the PBN matrix.

In [22]:
# test_PBN_matrix_7_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_7))
test_PBN_matrix_7_permuted = np.array([[ 26,   0,   0,   0,  59,   0,  49,   0,  29,   0,   0,   0,   0,   9,   0,  46],
                                       [  0,   0,  39,  17,   0,   0,   0,  49,  49,   0,   0,   0,   0,  98,  54,  17],
                                       [  0,   0,  26,  49,   0,   0,   0,   9,   0,   0,   0,  59,   9,   0,   0,   0],
                                       [  0,   0,   0,   0,   0,   9,  54,   0,   0,   0,   0,   0,  39,   0,  49,  26],
                                       [ 49,   0,   9,   0,   0,  29,   0,   0,   0,   0, 108,  63,   0,  17,  59,   0],
                                       [  0,   0,   0,  26,   0,   0,   0,  17,  39,  29,   0,   0,   0,   0,   0,  88],
                                       [ 17,  63,  88,   0,   0,   0,   0,  98,   0,   9,  37,   0,  88,   0,   0,  49],
                                       [  0,   0,   0,   0,  49,   0,   0,   0,  17,  37,   9,  29,  63,  49,  39,   0],
                                       [ 98,   0,   0,   0,  46, 108,  26,   0,   0,   0,   0,   9,   0,  63,   9,   0],
                                       [  0,  29,   0,   0,  17,   0,  29,   0,  46,   0,  26,   0,   0,   0,  26,   0],
                                       [ 29, 108,   0,  88,   0,  39,   0,   0,   0,   0,  39,   0,   0,  29,   0,   0],
                                       [  0,  39,   0,  39,   0,   0,   0,   0,  26,  26,  29,  39,   0,   0,   0,   0],
                                       [  0,   0,   0,   0,  29,   0,  98,  29,   0,  39,   0,   0,   0,   0,   0,   0],
                                       [ 46,   0,  49,  37,  39,  26,   9,   0,   0, 108,  17,   0,  49,   0,  29,   0],
                                       [  0,   9,  54,   9,   0,  54,   0,  37,   0,  17,   0,  49,  17,   0,   0,  39],
                                       [  0,  17,   0,   0,  26,   0,   0,  26,  59,   0,   0,  17,   0,   0,   0,   0]])


##### Execute GER on the permuted PBN matrix.

In [24]:
GER(test_PBN_matrix_7_permuted, 10)

(8,
 [49, 39, 59, 29, 26, 17, 37, 9],
 array([[ 4, 10, 13,  2,  7,  8,  0,  1,  1, 13,  4, 14, 13,  7,  3,  6],
        [ 8, 11,  1, 11, 13, 10, 12,  6,  5, 12, 10, 11,  3,  1,  7, 14],
        [ 8, 10,  6, 10,  0,  8, 12,  6, 15, 13,  4,  2,  6,  1,  4,  5],
        [10,  9,  6, 10, 12,  4,  9, 12,  0,  5, 11,  7,  6, 10, 13,  5],
        [ 0,  6,  2,  5, 15, 13,  8, 15, 11, 11,  9,  4,  7,  8,  9,  3],
        [ 6, 15, 14,  1,  9, 14,  3,  5,  7, 14, 13, 15, 14,  4,  1,  1],
        [13,  6, 14, 13,  8, 14,  3, 14,  9,  7,  6,  4,  7,  8,  1,  0],
        [13, 14,  4, 14,  8,  3, 13,  2,  9,  6,  7,  8,  2,  0,  8,  0]]))

#### [6] Small Example PBN Matrix Mentioned in the GER paper Section 2.2

In [64]:
test3 = np.array([[1, 5, 6,  0], 
                  [4, 0, 2,  0], 
                  [5, 2, 0, 10], 
                  [0, 3, 2,  0]])
GER(test3, 10)

(4,
 [2, 5, 2, 1],
 array([[1, 2, 1, 2],
        [2, 0, 0, 2],
        [1, 3, 3, 2],
        [0, 3, 0, 2]]))

# Testing GER on the three PBN matrices from the MOMP paper.

### 1. First PBN matrix $P_1$

It can be proved that GER finds the sparsest possible decomposition for $P_1$.

In [11]:
MOMP_PBN_P1 = np.array([[1, 3, 2, 1], 
                        [2, 3, 2, 0], 
                        [0, 0, 6, 4], 
                        [7, 4, 0, 5]])
GER(MOMP_PBN_P1, 10)

(5,
 [1, 2, 2, 4, 1],
 array([[0, 0, 2, 0],
        [1, 0, 0, 2],
        [3, 1, 1, 2],
        [3, 3, 2, 3],
        [3, 1, 2, 3]]))

### 2. Second PBN matrix $P_2$

It can be proved that GER finds the sparsest possible decomposition for $P_2$.

In [12]:
MOMP_PBN_P2 = np.array([[1, 3, 2, 1, 0, 0, 0, 0], 
                        [2, 3, 2, 0, 0, 0, 0, 0], 
                        [0, 0, 6, 4, 0, 0, 0, 0], 
                        [7, 4, 0, 5, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 3, 2, 1],
                        [0, 0, 0, 0, 2, 3, 2, 0],
                        [0, 0, 0, 0, 0, 0, 6, 4],
                        [0, 0, 0, 0, 7, 4, 0, 5]])
GER(MOMP_PBN_P2, 10)

(5,
 [1, 2, 2, 4, 1],
 array([[0, 0, 2, 0, 4, 4, 6, 4],
        [1, 0, 0, 2, 5, 4, 4, 6],
        [3, 1, 1, 2, 7, 5, 5, 6],
        [3, 3, 2, 3, 7, 7, 6, 7],
        [3, 1, 2, 3, 7, 5, 6, 7]]))

### 3. Third PBN matrix $P_3$

In [13]:
MOMP_PBN_P3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                        [14, 31,  0, 50, 12, 13, 33,  6], 
                        [ 0,  8, 40, 25, 25,  0, 67,  0], 
                        [ 0, 15,  0,  0,  0,  8,  0,  0],
                        [ 0, 15, 30,  0,  0, 13,  0,  0],
                        [29, 31, 20,  0, 25, 29,  0, 39],
                        [ 0,  0,  0,  0, 38,  0,  0,  0],
                        [ 0,  0,  0, 25,  0, 33,  0, 55]])
GER(MOMP_PBN_P3, 2)

(11,
 [25, 8, 6, 4, 2, 25, 13, 4, 7, 5, 1],
 array([[0, 1, 2, 2, 2, 7, 1, 5],
        [1, 2, 0, 1, 1, 3, 1, 5],
        [1, 1, 2, 1, 6, 7, 2, 1],
        [5, 3, 2, 1, 1, 0, 2, 5],
        [0, 4, 0, 1, 6, 7, 2, 5],
        [5, 5, 4, 7, 5, 5, 2, 7],
        [0, 4, 5, 1, 6, 1, 2, 7],
        [0, 3, 2, 1, 6, 5, 2, 7],
        [0, 3, 5, 1, 6, 4, 2, 7],
        [0, 5, 4, 1, 6, 4, 2, 7],
        [0, 5, 2, 1, 6, 4, 2, 7]]))

# Testing GER on the PBN matrices from the projection-based gradient descent method (PBGDM) paper.

### 1. First PBN matrix $P_1$

It can be proved that GER finds the sparsest possible decomposition for $P_1$.

In [14]:
PBGDM_PBN_P1 = np.array([[1, 3, 5, 6], 
                         [0, 7, 0, 0], 
                         [0, 0, 5, 0], 
                         [9, 0, 0, 4]])
GER(PBGDM_PBN_P1, 10)

(4,
 [5, 1, 2, 2],
 array([[3, 1, 0, 0],
        [0, 0, 2, 0],
        [3, 0, 2, 3],
        [3, 1, 2, 3]]))

### 2. Second PBN matrix $P_3$

It can be proved that GER finds the sparsest possible decomposition for $P_3$.

In [16]:
PBGDM_PBN_P3 = np.array([[10,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 0, 10, 0, 0,  0], 
                         [ 0,  0,  0, 0,  0, 0, 0,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0, 10, 10, 0,  0, 5, 0,  0],
                         [ 0,  0,  0, 0,  0, 5, 0, 10]])
GER(PBGDM_PBN_P3, 10)

(4,
 [3, 2, 3, 2],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]]))

### 3. Third PBN matrix $P_4(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

It can be proved that GER finds the sparsest possible decomposition for $P_4(0.01)$.

In [30]:
d = 1
PBGDM_PBN_P4_d1 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])
GER(PBGDM_PBN_P4_d1, 10)

(5,
 [1, 30, 19, 30, 20],
 array([[3, 3, 3, 0, 3, 6, 4, 3],
        [0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]]))

It can be proved that GER finds the sparsest possible decomposition for $P_4(0.02)$.

In [21]:
d = 1
PBGDM_PBN_P4_d2 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])
GER(PBGDM_PBN_P4_d2, 10)

(5,
 [1, 15, 9, 15, 10],
 array([[3, 3, 3, 0, 3, 6, 4, 3],
        [0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]]))

It can be proved that GER finds the sparsest possible decomposition for $P_4(0.03)$.

In [20]:
d = 3
PBGDM_PBN_P4_d3 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])
GER(PBGDM_PBN_P4_d3, 10)

(5,
 [3, 30, 17, 30, 20],
 array([[3, 3, 3, 0, 3, 6, 4, 3],
        [0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]]))

It can be proved that GER finds the sparsest possible decomposition for $P_4(0.04)$.

In [23]:
d = 2
PBGDM_PBN_P4_d4 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])
GER(PBGDM_PBN_P4_d4, 10)

(5,
 [2, 15, 8, 15, 10],
 array([[3, 3, 3, 0, 3, 6, 4, 3],
        [0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]]))

### 4. Fourth PBN matrix $P_6(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

It can be proved that GER finds the sparsest possible decomposition for $P_6(0.01)$.

In [29]:
PBGDM_PBN_P6_d1 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d1[0:8, 0:8] = PBGDM_PBN_P4_d1
PBGDM_PBN_P6_d1[8:16, 8:16] = PBGDM_PBN_P4_d1

GER(PBGDM_PBN_P6_d1, 10)

(5,
 [1, 30, 19, 30, 20],
 array([[ 3,  3,  3,  0,  3,  6,  4,  3, 11, 11, 11,  8, 11, 14, 12, 11],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15]]))

It can be proved that GER finds the sparsest possible decomposition for $P_6(0.02)$.

In [32]:
PBGDM_PBN_P6_d2 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d2[0:8, 0:8] = PBGDM_PBN_P4_d2
PBGDM_PBN_P6_d2[8:16, 8:16] = PBGDM_PBN_P4_d2

GER(PBGDM_PBN_P6_d2, 10)

(5,
 [1, 15, 9, 15, 10],
 array([[ 3,  3,  3,  0,  3,  6,  4,  3, 11, 11, 11,  8, 11, 14, 12, 11],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15]]))

It can be proved that GER finds the sparsest possible decomposition for $P_6(0.03)$.

In [33]:
PBGDM_PBN_P6_d3 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d3[0:8, 0:8] = PBGDM_PBN_P4_d3
PBGDM_PBN_P6_d3[8:16, 8:16] = PBGDM_PBN_P4_d3

GER(PBGDM_PBN_P6_d3, 10)

(5,
 [3, 30, 17, 30, 20],
 array([[ 3,  3,  3,  0,  3,  6,  4,  3, 11, 11, 11,  8, 11, 14, 12, 11],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15]]))

It can be proved that GER finds the sparsest possible decomposition for $P_6(0.04)$.

In [35]:
PBGDM_PBN_P6_d4 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d4[0:8, 0:8] = PBGDM_PBN_P4_d4
PBGDM_PBN_P6_d4[8:16, 8:16] = PBGDM_PBN_P4_d4

GER(PBGDM_PBN_P6_d4, 10)

(5,
 [2, 15, 8, 15, 10],
 array([[ 3,  3,  3,  0,  3,  6,  4,  3, 11, 11, 11,  8, 11, 14, 12, 11],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15]]))